In [ ]:
# Setup
import sys
sys.path.insert(0, '../src')

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
plt.style.use('seaborn-v0_8-darkgrid')

from datetime import datetime, timedelta
import warnings
warnings.filterwarnings('ignore')

## 1. Data Loading

In [ ]:
# Generate synthetic data for demo
def generate_synthetic_data(n_days=500):
    np.random.seed(42)
    dates = pd.date_range(start='2022-01-01', periods=n_days, freq='D')
    
    # Price with trend + mean reversion
    returns = np.random.normal(0.0003, 0.02, n_days)
    price = 100 * np.exp(np.cumsum(returns))
    
    data = pd.DataFrame({
        'open': price * (1 + np.random.uniform(-0.005, 0.005, n_days)),
        'high': price * (1 + np.random.uniform(0, 0.015, n_days)),
        'low': price * (1 - np.random.uniform(0, 0.015, n_days)),
        'close': price,
        'volume': np.random.uniform(1e6, 1e8, n_days)
    }, index=dates)
    
    data['high'] = data[['open', 'high', 'close']].max(axis=1)
    data['low'] = data[['open', 'low', 'close']].min(axis=1)
    
    return data

data = generate_synthetic_data(500)
print(f"Data shape: {data.shape}")
data.head()

In [ ]:
# Plot price
fig, axes = plt.subplots(2, 1, figsize=(14, 8), sharex=True)

axes[0].plot(data.index, data['close'], label='Close Price', color='blue')
axes[0].set_title('Price Chart')
axes[0].set_ylabel('Price')
axes[0].legend()

axes[1].bar(data.index, data['volume'], color='gray', alpha=0.5)
axes[1].set_title('Volume')
axes[1].set_ylabel('Volume')

plt.tight_layout()
plt.show()

## 2. Feature Engineering

In [ ]:
from features.feature_engineer import FeaturePipeline, FeatureConfig

config = FeatureConfig(
    sma_periods=[10, 20, 50],
    rsi_period=14,
    macd_params=(12, 26, 9),
    bb_period=20,
    atr_period=14,
    lookback_periods=[5, 10, 20],
    frac_diff_d=0.3
)

pipeline = FeaturePipeline(config)
features = pipeline.fit_transform(data)

print(f"Features shape: {features.shape}")
print(f"\nFeatures generated:")
for col in features.columns:
    print(f"  - {col}")

In [ ]:
# Plot RSI and Bollinger Bands
fig, axes = plt.subplots(3, 1, figsize=(14, 10), sharex=True)

# Price with Bollinger Bands
ax1 = axes[0]
ax1.plot(data.index, data['close'], label='Close', color='blue')
if 'bb_upper' in features.columns:
    ax1.plot(features.index, features['bb_upper'], '--', color='red', alpha=0.5, label='BB Upper')
    ax1.plot(features.index, features['bb_lower'], '--', color='green', alpha=0.5, label='BB Lower')
ax1.set_title('Price with Bollinger Bands')
ax1.legend()

# RSI
ax2 = axes[1]
if 'rsi_14' in features.columns:
    ax2.plot(features.index, features['rsi_14'], color='purple')
    ax2.axhline(70, color='red', linestyle='--', alpha=0.5)
    ax2.axhline(30, color='green', linestyle='--', alpha=0.5)
ax2.set_title('RSI(14)')
ax2.set_ylim(0, 100)

# MACD
ax3 = axes[2]
if 'macd' in features.columns:
    ax3.plot(features.index, features['macd'], label='MACD', color='blue')
    ax3.plot(features.index, features['macd_signal'], label='Signal', color='red')
    ax3.bar(features.index, features['macd_hist'], color='gray', alpha=0.3, label='Histogram')
ax3.set_title('MACD')
ax3.legend()

plt.tight_layout()
plt.show()

## 3. Triple-Barrier Labeling (AFML)

In [ ]:
from labeling.afml_labeling import TripleBarrierLabeler, TripleBarrierConfig

label_config = TripleBarrierConfig(
    pt_sl_ratio=2.0,  # 2:1 reward/risk
    max_holding_period=10,
    volatility_lookback=20,
    min_return=0.001
)

labeler = TripleBarrierLabeler(label_config)
labels_df = labeler.fit_transform(data)

print(f"Labels shape: {labels_df.shape}")
print(f"\nLabel distribution:")
print(labels_df['label'].value_counts())

In [ ]:
# Visualize labels
fig, ax = plt.subplots(figsize=(14, 6))

ax.plot(data.index, data['close'], color='gray', alpha=0.5, label='Price')

# Color code by label
valid_labels = labels_df.dropna()
colors = {1: 'green', 0: 'gray', -1: 'red'}

for label, color in colors.items():
    mask = valid_labels['label'] == label
    indices = valid_labels[mask].index
    ax.scatter(indices, data.loc[indices, 'close'], 
               c=color, s=20, alpha=0.6, 
               label=f'Label {label}')

ax.set_title('Triple-Barrier Labels')
ax.legend()
plt.tight_layout()
plt.show()

## 4. Model Training with Purged K-Fold CV

In [ ]:
from models.train import ModelTrainer, TrainerConfig

# Align features and labels
common_idx = features.index.intersection(labels_df.index)
X = features.loc[common_idx].dropna()
y = labels_df.loc[X.index, 'label'].dropna()

# Final alignment
common_idx = X.index.intersection(y.index)
X = X.loc[common_idx]
y = y.loc[common_idx]

print(f"Training samples: {len(X)}")
print(f"Features: {X.shape[1]}")
print(f"\nLabel distribution:")
print(y.value_counts())

In [ ]:
# Train Random Forest with Purged K-Fold
trainer_config = TrainerConfig(
    model_type='random_forest',
    n_splits=5,
    purge_gap=5,  # 5-day gap between train/test
    n_jobs=-1
)

trainer = ModelTrainer(trainer_config)
results = trainer.train(X, y)

print(f"\nCV Scores: {results['cv_scores']}")
print(f"Mean: {results['cv_scores'].mean():.4f} ± {results['cv_scores'].std():.4f}")

In [ ]:
# Feature Importance
importance = pd.Series(
    results['model'].feature_importances_,
    index=X.columns
).sort_values(ascending=True)

fig, ax = plt.subplots(figsize=(10, 8))
importance.tail(15).plot(kind='barh', ax=ax)
ax.set_title('Top 15 Feature Importances')
ax.set_xlabel('Importance')
plt.tight_layout()
plt.show()

## 5. Backtesting

In [ ]:
from backtest.engine import BacktestEngine, BacktestConfig

# Generate predictions
predictions = results['model'].predict(X)
signals = pd.Series(predictions, index=X.index)

print(f"Signal distribution:")
print(signals.value_counts())

In [ ]:
# Run backtest
bt_config = BacktestConfig(
    initial_capital=100000,
    commission=0.001,
    slippage=0.0005,
    position_size=0.1
)

engine = BacktestEngine(bt_config)
bt_results = engine.run(data.loc[signals.index], signals)

print("\nBacktest Results:")
print("-" * 40)

if 'metrics' in bt_results:
    for key, value in bt_results['metrics'].items():
        print(f"{key}: {value}")

In [ ]:
# Plot equity curve
if 'equity_curve' in bt_results and len(bt_results['equity_curve']) > 0:
    equity = bt_results['equity_curve']
    
    fig, axes = plt.subplots(2, 1, figsize=(14, 8), sharex=True)
    
    # Equity curve
    axes[0].plot(equity.index, equity.values, color='blue')
    axes[0].set_title('Equity Curve')
    axes[0].set_ylabel('Portfolio Value ($)')
    axes[0].axhline(bt_config.initial_capital, color='gray', linestyle='--', alpha=0.5)
    
    # Drawdown
    running_max = equity.cummax()
    drawdown = (equity - running_max) / running_max * 100
    axes[1].fill_between(drawdown.index, drawdown.values, 0, color='red', alpha=0.3)
    axes[1].set_title('Drawdown')
    axes[1].set_ylabel('Drawdown (%)')
    
    plt.tight_layout()
    plt.show()
else:
    print("No equity curve data available")

## 6. Performance Summary

In [ ]:
from utils.metrics import PerformanceReport

if 'equity_curve' in bt_results and len(bt_results['equity_curve']) > 0:
    equity = bt_results['equity_curve']
    returns = equity.pct_change().dropna()
    
    report = PerformanceReport(returns, equity)
    metrics = report.generate_report()
    
    print("="*50)
    print("PERFORMANCE REPORT")
    print("="*50)
    
    for section, values in metrics.items():
        print(f"\n{section.upper()}:")
        if isinstance(values, dict):
            for key, val in values.items():
                if isinstance(val, float):
                    print(f"  {key}: {val:.4f}")
                else:
                    print(f"  {key}: {val}")
        else:
            print(f"  {values}")
else:
    print("Cannot generate report - no equity curve data")

## Next Steps

1. **Use Real Data**: Replace synthetic data with actual market data (CCXT, Yahoo Finance)
2. **Hyperparameter Tuning**: Use Optuna for automated optimization
3. **Walk-Forward Optimization**: Implement rolling window retraining
4. **Meta-Labeling**: Add secondary model for bet sizing
5. **Live Trading**: Connect to Delta Exchange API for execution